# SFT, PPO, DPO

In [1]:
from datasets import load_dataset, Dataset
from trl import SFTConfig, SFTTrainer, AutoModelForCausalLMWithValueHead
from transformers import GPT2Tokenizer, AutoModelForCausalLM, GenerationConfig, AutoTokenizer
import pandas as pd
import torch, trl

folio = load_dataset("yale-nlp/FOLIO")

folio_test = folio['validation']
folio_train = folio['train']
folio_train
#p_folio = load_dataset("yale-nlp/P-FOLIO") # HABLARLES A LOS DE YALE

Dataset({
    features: ['story_id', 'premises', 'premises-FOL', 'conclusion', 'conclusion-FOL', 'label', 'example_id'],
    num_rows: 1001
})

### Cleaning FOLIO for SFT training

**v0.1** We remove the 'Uncertain' tag and carry out the SFT using only True (1) and False (0) tags.

In [2]:
clean_premises = []
clean_labels = []
for i in range(len(folio_train['label'])):
    if folio_train['label'][i] == 'True':
        clean_labels.append(1)
        clean_premises.append(folio_train['premises'][i])
    elif folio_train['label'][i] == 'False':
        clean_labels.append(0)
        clean_premises.append(folio_train['premises'][i])


dict_dataset = {'text': clean_premises, 'label': clean_labels}
folio_df = pd.DataFrame(dict_dataset)
folio_new = Dataset.from_pandas(folio_df)

folio_new

Dataset({
    features: ['text', 'label'],
    num_rows: 677
})

### SFT Training

In [3]:
# Basic configuration following: https://huggingface.co/docs/trl/v0.16.1/sft_trainer

#model = AutoModelForCausalLM.from_pretrained('openai-community/gpt2')

#training_args = SFTConfig(
#    learning_rate = 1e-05,
#    weight_decay = 0.02,
#    num_train_epochs = 2.0
#)

#trainer = SFTTrainer(
#    model,
#    train_dataset = folio_new,
#    args = training_args,
#)

In [26]:
#%%time
# Only train if working on a new model
#trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


CPU times: total: 25.9 s
Wall time: 27.2 s


TrainOutput(global_step=170, training_loss=2.5084853228400736, metrics={'train_runtime': 27.0288, 'train_samples_per_second': 50.095, 'train_steps_per_second': 6.29, 'total_flos': 94077889920000.0, 'train_loss': 2.5084853228400736, 'epoch': 2.0})

In [27]:
# Only push if there is a new model.
#model.push_to_hub("gpt2-sft-0.1")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kurosawama/gpt2-sft-0.1/commit/c307f0abdef75926a12dbfea9f6625ddbc49f1db', commit_message='Upload model', commit_description='', oid='c307f0abdef75926a12dbfea9f6625ddbc49f1db', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Kurosawama/gpt2-sft-0.1', endpoint='https://huggingface.co', repo_type='model', repo_id='Kurosawama/gpt2-sft-0.1'), pr_revision=None, pr_num=None)

### DPO

In [4]:
ex_dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")

In [5]:
#ex_dataset['chosen'][0]

In [6]:
#print("Tipo de dato de 'chosen': {}. Longitud de 'chosen': {}.".format(type(ex_dataset['chosen'][0]), len(ex_dataset['chosen'][0])))

In [7]:
#ex_dataset['rejected'][0]

In [8]:
#print("Tipo de dato de 'chosen': {}. Longitud de 'chosen': {}.".format(type(ex_dataset['rejected'][0]), len(ex_dataset['rejected'][0])))

In [9]:
#print(ex_dataset['score_chosen'][0], ex_dataset['score_rejected'][0])

**Datasets para DPO**

Tenemos que modificar nuestro dataset para que contenga 4 características:
1. 'chosen'
2. 'rejected'
3. 'score_chosen' (float)
4. 'score_rejected' (float)

Chosen y rejected son listas con dos elementos. Los elementos de esta lista corresponden a diccionarios en formato de prompt y contenido para un modelo de lenguaje:
1. {'content': texto, 'role': 'user'}
2. {'content': respuesta, 'role': 'assistant'}

In [10]:
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(dev)
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

cuda
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 

In [11]:
%%time
from trl import DPOConfig, DPOTrainer

# LLAMA ES MUY PESADO ME LLEVA LA CHINGADA ODIO SER POBRE

#llama_id = 'meta-llama/Llama-3.1-8B'
#llama_id = 'meta-llama/Llama-3.2-8B'
#llama_model = AutoModelForCausalLM.from_pretrained(llama_id).to(dev)
#llama_tokenizer = AutoTokenizer.from_pretrained(llama_id)
#llama_tokenizer.pad_token = llama_tokenizer.eos_token
#llama_tokenizer.chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
#{{ system_prompt }}<|eot_id|><|start_header_id|>user<|end_header_id|>
#{{ user_msg_1 }}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
#{{ model_answer_1 }}<|eot_id|>"""


model = AutoModelForCausalLM.from_pretrained('openai-community/gpt2')
tokenizer = AutoTokenizer.from_pretrained('openai-community/gpt2')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = """
    <|im_start|>system
    {SYSTEM}<|im_end|>
    <|im_start|>user
    {INPUT}<|im_ed|>
    <|im_start|>assistant
    {OUTPUT}<|im_end|>
"""

training_args = DPOConfig(output_dir = r'C:\Users\FLopezP\Desktop\PCIC\Tesis', logging_steps = 10)

trainer = DPOTrainer(model=model, args=training_args, processing_class=tokenizer, train_dataset=ex_dataset)
trainer.train()

Step,Training Loss
10,0.693100
20,0.693100
30,0.693100
40,0.693100
50,0.693100
60,0.693100
70,0.693100
80,0.693100
90,0.693100
100,0.693100


KeyboardInterrupt: 

### PPO
The SFT model will be used as a reference model for PPO.

In [23]:
# Parámetros necesarios para implementar PPO mediante TRL
model = AutoModelForCausalLM.from_pretrained('openai-community/gpt2')
model = AutoModelForCausalLMWithValueHead.from_pretrained(model)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained('Kurosawama/gpt2-sft-0.1')
tokenizer = GPT2Tokenizer.from_pretrained('openai-community/gpt2')

In [24]:
# Tests to get PPOTrainer to work
tokenizer2 = AutoTokenizer.from_pretrained('openai-community/gpt2')
generation_config = GenerationConfig.from_pretrained('openai-community/gpt2')
model.generation_config = generation_config
ref_model.generation_config = generation_config

In [25]:
config = PPOConfig(
    learning_rate=1.47e-5,
    num_train_epochs=3.0,
    mini_batch_size=4,
    batch_size=16,
    gradient_accumulation_steps=1,
)

In [26]:
from trl import PPOConfig, PPOTrainer

ppo_trainer = PPOTrainer(
    config,
    model, 
    ref_model, 
    tokenizer2, 
    train_dataset = folio_new, 
    reward_model = ref_model
)

AttributeError: GPT2TokenizerFast has no attribute modules

In [ ]:
# PROBLEMA: EL TOKENIZADOR NO TIENE 'attribute modules'